# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [3]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    current_height = browser.execute_script("return document.body.scrollHeight;")
    
    if current_height == last_height:
        print('到達頁面底端')
        break
    else:
        print('網頁更新中...')
        last_height = current_height
        continue
   

網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
到達頁面底端


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [4]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [5]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 新台幣開盤貶1.8分後反彈　再度升破30元關卡

▲ 新台幣兌美元今天開盤貶值1.8分。（圖／路透社）
記者李瑞瑾／台北報導
台北外匯市場新台幣兌美元匯率今（14）日開盤貶值1.8分、以30.05元兌1美元開出。
新台幣兌美元昨（13）日收在30.037兌1美元；今天開盤貶值1.8分，以30.05元兌1美元開出。
新台幣開盤之後轉為走升，並再度升破30元關卡。截至上午9點08分，新台幣來到29.988元兌1美元，升值4.9分。
----------------------------------------------------------------------

[1] 台股開盤小漲14點　站上11,800點狹幅震盪

▲ 台股。（資料照／記者湯興漢攝）
記者李瑞瑾／台北報導
美股自高點回落，道瓊昨日收盤跌逾百點，那指、標普亦收低。台股今（14）日開盤則上漲14點、以11,806點開出。
截至上午9點03分，加權指數來到11,807點、上漲15點。
電子三雄中，台積電（2330）以337元開出、上漲0.59%；鴻海（2317）開在平盤84.7元；股王大立光（3008）也以平盤4820元開出。
美國新增華為商業秘密盜竊指控，抑制半導體類股漲勢，另外，紐約聯邦準備銀行宣布週五起將限縮Repo操作，更拉大美股尾盤跌幅，美股四大指數漲跌互見，道瓊跌逾百點，那指、標普收低，僅費半微幅走揚。
美股道瓊指數下跌128.11點，或0.43%，收29,423.31點；那斯達克指數下跌13.99點，或0.14%，收9,711.97點；標普500指數下跌5.51點，或0.16%，收3,373.94點；費城半導體指數上漲1.52點，或0.078%，收1,960.34點。
----------------------------------------------------------------------

[2] 美股道瓊跌128點　台指期開盤下跌18點

▲美股道瓊下跌 128.11 點。回頭看台指期今（14）日開盤下跌 18 點，來到 11,757 點。（圖／路透）
記者陳心怡／台北報導
新冠肺炎（COVLD-19）疫情的消

▲藝文場館照常開放，但許多表演節目受到疫情衝擊，面臨延期、取消、觀眾退票等情形。（資料照／記者屠惠剛攝）
記者林育綾／台北報導
新冠肺炎疫情影響，不少國內藝文產業也受到衝擊，包括演出取消、延期或退票等狀況。文化部近期已研議藝文產業短期紓困措施、中長期振興因應方案，建議行政院納入移緩濟急及後續特別預算支應項目。
文化部長鄭麗君自春節起已召開多次會議，除訂定文化部所屬場館防疫注意事項外，近期也研議藝文產業短期紓困措施、中長期振興因應方案。行政院院會今（13）日討論內需、觀光及運輸、農業產業之紓困方案，鄭麗君也在會中也補充說明藝文產業受疫情衝擊之樣態，並提出藝文產業短期紓困及中長期振興需求，建請納入移緩濟急及後續特別預算支應項目。
鄭麗君指出，藝文產業因疫情導致相關消費減少，衝擊樣態包括：表演藝術、傳統藝術演出取消、延期、退票，視覺藝術及工藝產業受影響，出版業及書店損失，電影票房及影視製作損失、流行音樂展演受影響、國內外展會延期，另外也有微型及小型文創業發生周轉及營運困難等。
----------------------------------------------------------------------

[14] 國銀海外市場2019年大賺732億創史上新高　香港市場是海外金雞母

▲根據金管會統計，2019年國銀稅前盈餘為732.6億元，一舉創下歷史新高。（圖／pakutaso）
記者紀佳妘／台北報導
金管會今（13）日公布國銀海外分支機構獲利，2019年稅前盈餘為732.6億元，一舉創下史上新高，其中香港市場大賺389.8億元，較上年成長近17%，主要是放款利息收入、投資收益增加所致，同步改寫歷史新高紀錄。
觀察海外市場獲利前10大國銀表現，香港去年獲利389.8億元，較上年同期成長16.9%，成為最賺錢的海外市場；排名第二的是大陸69.8億元，年增15.5%；排名第三的是美國61.1億元，較上年同期減少22.8%；第四則為日本獲利59.1億元，年增14%。
越南以獲利33.2億元排名第五，較上年成長45.6%，其餘第6至10名依序為新加坡（獲利32.3億元）、柬埔寨（獲利27.3億元）、英國（獲利14.2億元）、澳洲（獲利13.9億元）、加拿大（獲利6.1億元）。
美國去年獲利年減逾2成，金管會銀行局副局長莊琇媛表示，由於去年美國降息，使得利息淨收益減

非經濟重大事件對金融市場干擾不超過五個月，但能夠克服致命恐懼，危機入市的少之又少，股市反映未來3-6個月後市，從人民幣匯率與陸股不難看出端倪。
文／馮泉富
一周過去了，所有媒體仍是一面倒的負面新聞，雖然大陸官方不斷增加支援武漢的醫療力量，總計支援武漢的醫護人員已達一萬一千多人，可是疫情仍舊持續蔓延，累積的確診案例一路增加。
疫情干擾效應降低  亞股市場反彈築底
從封城進展到封省，對外隔離政策並一路擴散至廣州、深圳以及直轄市天津等一線城市，連中國大陸最大的兩大政經中心，北京、上海，所有社區都實施封閉式管理，等於四大一線城市「北上廣深」、四大直轄市「京津滬渝」全數進入半封城狀態，這些人口最密集、經濟活動最熱絡的幾大城市陸續提升管制措施，使各行業職工返回工作崗位難度大增。
根據百度遷徙路徑規模指數顯示，到達湖北省人數最多的城市依序正是深圳、廣州，返程人員往往又面臨十四天隔離，這勢必也影響眾多台商企業的復工進度，更別說後續缺工、缺料的問題，市場更擔憂復工之後的交叉感染大爆發。
以上的陳述及問題都是事實，我們並不否認。不過，根據過去經驗，這類重大事件對金融市場的干擾影響僅大約五個月不到。
事實上金融市場之走勢與訊息面的關聯，本來就經常不同步，這就是金融市場，否則，這麼簡單的話，所有人看新聞做股票就好了，還需要學習嗎？本次的事件就是經典範例。
很多人嘴巴都會說尊重市場，是尊重人嗎？錯，該尊重的是市場資金的想法，而不是人頭，這與投票行為，男女老少票票等值不同，這就是金融市場殘酷的地方。
先前已提醒過，與其跟著新聞自己嚇自己，投資人更應該留意追蹤中國權值股的新加坡A50期貨之走勢，因為這是全球法人參與的市場，具有高度代表性。
另外，若真要剖析疫情與股市行情的關聯性，也不是在確診人數，而是在確診人數之增速，這樣才不至於人云亦云，過度恐慌。由於美股已領先創下歷史新高，亞股現階段回歸產業基本面的角度來觀察，相對比較不易犯錯。
美股領先創新高  台股進入修復期
不過，我們的看法是，只要全球都很重視這個事件，金融市場反而大概率不會出事。近期美國的經濟數據指標顯示，消費者支出和服務業活動持續給予經濟支撐。
就日前公布的一月非農新增就業人數，達二二．五萬人，高於華爾街預估的十五．八萬人，亦較去年十二月經上修後的十四．七萬人大幅回升；勞動參與率上升○．二個百分點，來到六三．四％，創下二○一三年六月

▲精材董事長陳家湘（右）。（圖／記者周康玉攝）
記者周康玉／台北報導
台積電轉投資的半導體晶圓封裝廠精材（3374）今（13）日舉行法說會。精材董事長陳家湘表示，中美貿易戰還未底定，加上冠狀病毒疫情升溫，讓大家很難放心，展望全年，上半年從訂單來評估「審慎樂觀」，下半年則要看大環境情勢轉變而定。
精材去（2019）年因感測元件封裝需求大增，在連續虧損三年後，由虧轉盈，股價也從去年中31元的波段低點，一路步步高升，短短半年三級跳至90元以上，問鼎「百元俱樂部」。今日股價最高來到95.6元，漲4.8%，創兩年三個月以來新高；開高走低，終場以92.2元坐收，跌1.7元或1.18%。
精材去年第4季合併營收13.96億元，季減16.4%、年減2.2%；合併毛利率21.5%，較去年第3季的26.6%減少5.1個百分點，較前年同期增加6.4個百分點；合併營業利益率15.5%，較去年第3季22%減少6.5個百分點，比前年同期9.7%增加5.7個百分點；稅後淨利為1.96億元，年減44.1%，比前年同期1.27億元增加54.4%。去年第4季每股稅後純益0.72元，低於去年第3季EPS1.29元，優於前年同期EPS 0.47元。
精材去年全年合併營收46.53億元，年減1.3%；合併毛利率11.8%，年增11.6個百分點，去年營益率較前年轉正達到5%；稅後淨利1.82億元及每股稅後純益0.67元，較前年每股大虧4.99元轉盈。
陳家湘表示，去年上半年營收達15.87億元、淨損3.65億元，處於虧損狀態，主要因智慧型手機成長停滯，相關封裝服務營收驟減；與此同時，12吋產線於減損折舊成本降低，有效縮減營運虧損。
然而下半年因終端客戶發表新手機，帶動3D感測訂單回升，加上生產管理與製程改進，製造成本顯著降低等因素，下半年營收回升，全年由虧轉盈，精材去年整年表現優於預期。
展望今（2020）年上半年，陳家湘表示，今年全球大環境遭遇更多變數，中美貿易戰還未底定，加上冠狀病毒疫情升溫，讓大家很難放心，上半年從訂單來評估「審慎樂觀」，需求會比去年同期增加，季節性差異會縮小，下半年還在看整個國際情勢和疫情後續影響。
陳家湘表示，因智慧手機多鏡頭滲透率提升，CMOS影像感測器（CIS）晶圓級尺寸封裝（WLCSP）訂單會比去年同期好一些，但受限產能正逐步轉往高毛利率產品，產能也有限，但仍會優於去年。
精材

▲台灣三星認為，即便武漢肺炎疫情延燒，但不會影響該公司在台灣第一季的供貨表現。（圖／記者邱倢芯攝）
記者邱倢芯／台北報導
三星電子近期推出了最新Galaxy S20系列與Galaxy Z Flip等機款，不過有不少台灣消費者擔心，三星剛好在新冠肺炎疫情發生的時間點推出新機，是否會影響在台灣的供貨？對此，台灣三星副總經理陳啟蒙指出，面對這次的疫情，三星已經做好充分準備，因此可以跟消費者保證第一季供貨不會有影響。
陳啟蒙表示，台灣這邊的銷售會與總部爭取最大支援，就目前為止都是正常進貨中，短時間來說，三星對於台灣市場有一定的信心，因此會繼續供貨；就長時間來說，則是希望這波疫情可以早日結束。
另一方面，台灣三星在今（13）日宣布Galaxy Z Flip將於2/21正式在台灣上市，相較於前一代的摺疊手機Galaxy Fold，Galaxy Z Flip台灣上市的時間提前相當多，也讓人不免好奇為什麼這次會提前這麼多？
陳啟蒙指出，先前Galaxy Fold在台灣上市後，以相當快的速度銷售完畢，讓三星對於摺疊手機在台銷售具有相當的信心，雖然這次台灣仍算是Galaxy Fold第二波上市的國家，但事實上與第一波上市國家也僅差了7天，由此可知總部對於台灣銷售具有信心。
----------------------------------------------------------------------

[39] 群創元月營收166億元　較上月減少24.9%

▲群創。（圖／記者姚惠茹攝）
記者姚惠茹／台北報導
群創（3481）今（13）日公布2020年元月自結合併營收為新166億元，較上月減少24.9%，較去年同期減少21.8%。
群創表示，2020年1月大尺寸合併出貨量共計888萬片，較上月減少21.3%；中小尺寸合併出貨量共計2,164萬片，較上月減少25.9%。
中國大陸廠復工方面，群創指出，寧波、佛山、上海三廠區已經在2月10日復工，並將啟動彈性辦公替代方案以降低相關影響，視疫情後續發展情況及當地政府政策做適當調整，而且已與客戶及供應商密切聯繫，調整出貨時間安排，預計將在復工後儘速滿足客戶出貨需求。
群創強調，惟因工作天數減少，將影響2月營收表現，實際影響情況仍須視疫情後續發展以及復工狀況而定。
-----------------------------------

▲行政院今（13）日將拍板發放「振興抵用券」拚經濟，經濟部將針對餐飲、零售、會展、商圈、夜市及傳統市場等五大內需服務業需求，提出紓困方案。（圖／記者湯興漢攝）
記者林淑慧／台北報導
新冠肺炎疫情延燒，重創各行各業，行政院今（13）日將拍板發放「振興抵用券」拚經濟，經濟部表示已盤點餐飲、零售、會展、商圈、夜市及傳統市場等五大內需服務業需求，並匡列上百億元的特別預算支應，展現政府力挺服務業與中小企業的決心。
新冠肺炎疫情衝擊國內產業，行政院會今天將通過經濟內需型產業、農業、交通相關產業的紓困、輔導、復甦與振興方案，依據內部規劃，待疫情結束後，也將在資金面提供業者新增貸款的能量、在市場面透過大型展銷活動來吸引消費意願回籠，並透過振興抵用券等相關措施，刺激民生消費。
針對台灣服務業可能被武漢肺炎疫情而影響，據了解，振興抵用券的細節規劃包括比照去年夜市抵用券的成功經驗，由年度公務預算編列，為達「有感」政策，預算規模可望優於夜市券，「上看10億元」。
另一種做法則是產業界期盼的紓困措施。經濟部已就餐飲、零售、會展、商圈、夜市及傳統市場等五大項內需服務業，全面盤點疫情對其的影響及衝擊，將按特別條例提出紓困方案，預算規模將超過百億元，待行政院會拍板後隨即著手研擬。
官員透露，振興抵用券是吸取去年度夜市券的成功經驗，加以擴大調整。在使用範圍上，目前規劃為服務業（如夜市、商圈等）。使用方式則比照夜市券，採取消費一定金額可以部分抵用，以期帶動民生消費，創造額外效益。
----------------------------------------------------------------------

[51] 鴻海重訊聲明：路透報導中國廠區復工時程「並非事實」　

▲富士康。（圖／路透）
記者姚惠茹／台北報導
新冠肺炎（COVID-19）疫情延燒，鴻海（2317）今（13）日重訊公告，針對近期路透社（Reuters）報導鴻海集團旗下富士康中國大陸廠區的復工時程規劃並非事實，並對此保留法律追訴權。
鴻海指出，近日有關路透社報導，集團於中國大陸廠區整體復工時程規劃並非事實，鴻海特此澄清，以正視聽。
鴻海表示，過去這段時間該媒體有關集團中國大陸廠區復工訊息，多屬不實，盼媒體勿再捕風捉影造成員工、投資人、社會大眾誤會及困擾。公司對此保留法律追訴權。
-------------------

▲經濟部今（12）日公告口罩60條產線分配結果。（圖／經濟部提供）
記者林淑慧／台北報導
武漢肺炎疫情持續延燒，近期口罩一罩難求，政府斥資自建口罩產線，經濟部今（12）日公告60條產線分配結果，包括康匠、永猷、恒大等15家口罩業者助攻，預計三月起力拚千萬片產能。
依據經濟部規劃，國內口罩未來每週日產能可增加70萬片，兩週後可望達到460萬片。隨著60台新機具投入生產後，3月上旬產能可達到1000萬片。
經濟部工業局今天公佈60條口罩產線遴選結果，國內大廠如康那香等15家廠商入列。其中，康匠、永猷、恒大各獲得10條產線，康那香、順益利獲得5條，舜堡獲得4條，宣德與格安德各獲得3條。
其次，香港商麥迪康、淨新與易廷各獲2條產線；威欣利、加利、奕綸與昭惠各獲得1條產線。依據地區別觀察，北部地區以八家為最多，南部地區有五家，中部地區則有二家。
官員指出，上述口罩廠商須於下月2日前完成新機台安裝並開始生產，每一新機台每週產能至少70萬片口罩，每一新機台總產量至少500萬片口罩；其中，500萬片中的120萬片政府不收購，為廠商必須提供；其餘的380萬片則依照目前的徵用價格徵用。
經濟部表示，原來需要半年工期的口罩產線，因有工具機、機械業者的主動協助，希望可以用台灣世界級的技術跟零組件，支援政府打造產線，工期壓縮在一個月內全部完工，堪稱「不可能的任務」，助攻國內口罩日產能自三月起上看千萬片。
----------------------------------------------------------------------

[61] 富邦人壽祭出「GOLD」4策略迎戰2020年　整合資源、拓展海外市場

▲展望2020年，富邦人壽以「GOLD」四大戰略，整合資源專業成長、洞察趨勢掌握商機、暖心關懷回饋社會及拓展海外穩健發展。（圖／資料照）
記者紀佳妘／台北報導
受到武漢肺炎（新冠病毒，COVlD-19）疫情影響，富邦人壽今（12）日取消原定春酒行程，不過展望2020年，富邦人壽總經理陳俊伴指出，外在環境紛擾尚在，壽險業仍有嚴峻試煉，今年將以「GOLD」四大戰略迎戰。
在中美貿易戰、地緣政治動盪、低利率趨勢、監理力道加大等影響下，富邦人壽總經理陳俊伴表示，2019年對壽險業而言，業務發展及財務策略面臨高度挑戰，然而挑戰也帶來契機，更是保險公司回應轉變、強化本質、展現實力的

▲軟體銀行大受願景基金影響，淨利表現嚴重下跌。（圖／路透社）
記者謝仁傑／綜合報導
據國外媒體報導，日本的軟銀集團週三表示，在10月至12月這一季度的集團營業利潤為26億日圓（約新台幣6.85億元），遠低於去年同期的4380億日圓，減幅高達99％，主要是因為關係企業願景基金（Vision Fund）大虧2250億日圓（約新台幣614.85億元）。
願景基金虧損金額，遠高於去年同期的1760億日圓，拖累了軟銀集團。在提交給日本證交所的檔案中顯示，軟銀集團在10月至12月的這一季度，集團營業利潤為26億日圓（約新台幣6.85億元），遠低於三名分析師的平均預期值3450億日圓。
財報公佈之際，正值孫正義努力為願景基金尋找外部資金之時。上一季願景基金就已經出現危機，最主要因為WeWork估值大幅下跌，導致虧損89億美元（約新台幣2634億元）。
儘管軟銀今日股價因Sprint和T-Mobile的合併成功，飆到7個月來的最高點；執行長孫正義也為公司業績辯護，指出近期軟銀及願景基金所投資的Uber，估值正在增長。
願景基金表示，截至12月底，投資了88家公司，總投資額為746億美元，價值已提高至798億美元。
----------------------------------------------------------------------

[73] 啥米！期限已過　尚有46家境外電商沒開發票 eBay、NBA都在內

▲eBay。（圖／達志影像／美聯社）
記者吳靜君／台北報導
財政部要求只要在我國開立稅籍的境外電商，今年1月起都要開發票，眾所周知，大型的境外電商如Goole、Apple等都已經在去年陸續開立；不過仍有46家境外電商仍未開發票，包含eBay、Adobe、NBA等都在名單內。
根據財政部統計資料顯示，目前在我國設立稅籍、且營業中的境外電商有112家，已經開立的境外電商僅有66家，剩餘46家未開立；財政部台北國稅局指出，目前有和稅務代理人聯繫上的有28家，剩下18家無法取得聯繫，或者被「已讀不回」。
而有聯繫上比較知名的企業包含Adobe、eBay、Vimeo等，這些企業皆表示，因為因應我國的稅務獨特性，系統調整時間比較久，所以才會超時，目前系統還在測試中；但也有知名企業無法取得聯繫，像是NBA、日經等。國稅局官員表示，除了與稅務代理人聯繫外，國稅局也試圖

▲左起為彥臣董事長黃中洋、國際眼藥大師邱春億。（圖／彥臣提供）
記者姚惠茹／台北報導
彥臣生技（4732）今（12）日公布，2019年業績狂飆，全年營收為3.15億元，較2018年的1.08億元，成長近300%，創歷史新高紀錄，並在2019年12月成功完成現金增資，股價更是從年初順勢搭上防疫順風車，展現強勁的氣勢，一度突破2015年來高點。今日收在29.19元，跌幅0.75%，守在區間整理。
彥臣生技表示，新冠肺炎（武漢肺炎，COVID-19）疫情延燒，帶動新藥及醫藥保健品的業績，帶動彥臣同步受惠，股價從1月中就開始上揚，年前封關一度漲到30元，突破2015年來高點，年後開春更是一度超過31元，近期則守在29-30元區間整理，今日收在29.19元，跌幅0.75%。
黃中洋指出，彥臣趁勢加速國外專利運作，從去年至今年2月，用於治療眼疾的化合物與天然萃取物接連取得澳洲、韓國、日本及歐洲等發明專利，儲備未來多元化營收的資源。
黃中洋分享，彥臣持續專注於小分子新藥研發，以產業鏈分工的模式，大幅縮短研發時程，國際熱門標靶抗癌藥物「HDAC抑制劑」已獲得多國專利；腦癌新藥NBM-BMX也於2018年通過美國FDA審查，持續進行人體試驗中。
----------------------------------------------------------------------

[87] 「疫情將使GDP下修0.35-0.5個百分點」主計長證實　全年經濟成長率將下修至2.37%　

▲主計長朱澤民（右）。（圖／記者余弦妙攝）
記者余弦妙／台北報導
行政院主計總處今（12）日公布下修2020全年經濟成長率至2.37%，調降0.35個百分點，每人GDP為2萬7,437美元，CPI上漲0.62％。
主計總處最新預測，2020年GDP從2.72%下修0.35個百分點至2.37%。今年逐季表現，第1季受疫情影響最重，僅剩1.8%，第2季2.5%，第3季為全年最高2.75%，第4季基期較高，主計總處預測為2.4%。
主計長朱澤民表示，若是沒有疫情影響，原先預估今年的GDP應該可以在更高，但由於疫情干擾導致GDP必須要下修，不過原先預估下修幅度應該會比0.35個百分點高，但這次不會比SARS的衝擊大，且日期也不會超過3個月，因此預估下修的幅度會落在0.35-0.5個百分點這個區間。
不過，